In [1]:
#----Download dati esempio e apertura slicer con volume----#
import JupyterNotebooksLib as slicernb
import slicer, qt, ctk, time
from pathlib import Path

#Di default l'app si apre con il layout a 4 viste
slicer.app.layoutManager().setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView)
#Aggiunta sistema di riferimento
viewNodes = slicer.util.getNodesByClass("vtkMRMLAbstractViewNode")
for viewNode in viewNodes:
    viewNode.SetOrientationMarkerType(slicer.vtkMRMLAbstractViewNode.OrientationMarkerTypeAxes)

#----Personalizzazione markups----#
#Cambiare dimensione markups
defaultMarkupsDisplayNode = slicer.vtkMRMLMarkupsDisplayNode()
defaultMarkupsDisplayNode.SetGlyphScale(2.5)
defaultMarkupsDisplayNode.SetOpacity(0.9)
defaultMarkupsDisplayNode.SetGlyphTypeFromString("Sphere3D")
#Interactive Handles per traslazioni e rotazioni dei punti
defaultMarkupsDisplayNode.SetHandlesInteractive(True) 
defaultMarkupsDisplayNode.SetTranslationHandleVisibility(True)
defaultMarkupsDisplayNode.SetRotationHandleVisibility(True)
slicer.mrmlScene.AddDefaultNode(defaultMarkupsDisplayNode)
#Interactive Handles per rotazioni e traslazioni dei piani
planeHI = slicer.vtkMRMLMarkupsPlaneDisplayNode()
planeHI.SetHandlesInteractive(True)
planeHI.SetTranslationHandleVisibility(True)
planeHI.SetRotationHandleVisibility(True)
planeHI.SetOpacity(0.9)
slicer.mrmlScene.AddDefaultNode(planeHI)

#----Funzione per il salvataggio----#
saveWindow = qt.QDialog()
saveLayout = qt.QVBoxLayout()
saveCheckBox = qt.QCheckBox('Confermare la corretta segmentazione')
textInfo = qt.QLabel('Cliccando sul tasto save verrà salvata la scena in formato .mrb e verranno esportate le coordinate dei punti dei piani')
saveButton = qt.QPushButton('Save')
saveDynamicSpacer = ctk.ctkDynamicSpacer()
saveWindow.setModal(True)
saveButton.setEnabled(False)
textInfo.setWordWrap(True)
textInfo.setAlignment(8)
saveLayout.addWidget(saveCheckBox)
saveLayout.addWidget(textInfo)
saveLayout.addWidget(saveDynamicSpacer)
saveLayout.addWidget(saveButton)   
saveDialog = qt.QMessageBox()
saveDialog.setModal(True)
exportDialog = qt.QFileDialog()
exportDialog.setFileMode(4)    
dirName = ''
def functionButtonSave():
    dirName = exportDialog.getExistingDirectory()  
    cont = 0
    volumes = slicer.mrmlScene.GetNodesByClass('vtkMRMLVolumeNode')
    if volumes.GetNumberOfItems() == 0:
        volumeName = 'Scene'
    else:
        for volume in volumes:
            volumeName = volume.GetName()
    fileName = volumeName + '-' + time.strftime('%Y-%m-%d')
    if dirName != '':
        while cont == 0:            
            nameDialog = qt.QInputDialog()
            nameDialog.setWindowTitle('File Name')
            nameDialog.setLabelText('File Name:')
            nameDialog.setTextEchoMode(qt.QLineEdit().Normal)
            nameDialog.setTextValue('%s'%fileName)
            nameDialog.setOkButtonText('Save')
            if nameDialog.exec_() == qt.QDialog.Accepted:
                fileName = nameDialog.textValue()
                sceneSaveFilename = dirName + '/' + str(fileName) + '.mrb'
                if Path(sceneSaveFilename).is_file():
                    messageBoxSave = qt.QMessageBox()
                    messageBoxSave.setText("The file: '%s' already exists. \nDo you want to replace it?:" %fileName)
                    replaceButton = qt.QPushButton('Yes')
                    renameButton = qt.QPushButton('No, rename')
                    cancelButton = qt.QPushButton('Cancel')
                    messageBoxSave.addButton(replaceButton, 0)
                    messageBoxSave.addButton(renameButton, 0)
                    messageBoxSave.addButton(cancelButton, 0)
                    mb = messageBoxSave.exec()
                    if mb == 0:
                        cont = 1
                        deleteTableFunction()
                        exportPointsFunction()
                        saveTableFunction(dirName, fileName)
                        if slicer.util.saveScene(sceneSaveFilename):
                            saveDialog.setText('Scene saved')
                            saveDialog.show()
                        else:
                            saveDialog.setText('Scene saving failed')
                            saveDialog.show()
                    elif mb == 1:
                        cont = 0
                    elif mb == 2:
                        cont = 1
                        nameDialog.close()
                else:
                    cont = 1
                    exportPointsFunction()
                    saveTableFunction(dirName, fileName)
                    if slicer.util.saveScene(sceneSaveFilename):
                        saveDialog.setText('Scene saved')
                        saveDialog.show()
                    else:
                        saveDialog.setText('Scene saving failed')
                        saveDialog.show()
                saveWindow.close()
            else:
                cont = 1
                saveDialog.setText('Scene saving failed')
                saveDialog.show()
                nameDialog.close()
                saveWindow.close()        
        
def functionButton(bool):
    if saveButton.enabled == False:
        saveButton.setEnabled(True)
        saveButton.connect('clicked()', functionButtonSave)
    else:
        saveButton.setEnabled(False)

#----Funzione per esportare i punti----#
def exportPointsFunction():
    planeNodes = slicer.mrmlScene.GetNodesByClass('vtkMRMLMarkupsPlaneNode')
    if (planeNodes.GetNumberOfItems())==0:
        j=0
    elif (planeNodes.GetNumberOfItems())==1:
        j=1
    elif (planeNodes.GetNumberOfItems())>1:
        j=(planeNodes.GetNumberOfItems())-1    
    for planeNode in planeNodes:        
        for i in range(0,j):
            tableNodeRAS = slicer.vtkMRMLTableNode()
            tableNodeRAS.AddColumn().SetName('Label')
            tableNodeRAS.AddColumn().SetName('r')
            tableNodeRAS.AddColumn().SetName('a')
            tableNodeRAS.AddColumn().SetName('s')
            tableNodeRAS.AddEmptyRow(); tableNodeRAS.AddEmptyRow(); tableNodeRAS.AddEmptyRow()
            arrayRAS = slicer.util.arrayFromMarkupsControlPoints(planeNode, True)      
            tableNodeLPS = slicer.vtkMRMLTableNode()
            tableNodeLPS.AddColumn().SetName('Label')
            tableNodeLPS.AddColumn().SetName('l')
            tableNodeLPS.AddColumn().SetName('p')
            tableNodeLPS.AddColumn().SetName('s')
            tableNodeLPS.AddEmptyRow(); tableNodeLPS.AddEmptyRow(); tableNodeLPS.AddEmptyRow()
            arrayLPS = slicer.util.arrayFromMarkupsControlPoints(planeNode, True)
            for row in range(0,3):
                for col in range(0,4):
                    if col==0:
                        tableNodeRAS.SetCellText(row,col,str(planeNode.GetNthControlPointLabel(row)))
                        tableNodeLPS.SetCellText(row,col,str(planeNode.GetNthControlPointLabel(row)))
                    elif col==1 or col==2:
                        tableNodeRAS.SetCellText(row,col,str(arrayRAS[row,col-1]))
                        tableNodeRAS.SetCellText(row,col,str(arrayRAS[row,col-1]))
                        tableNodeRAS.SetCellText(row,col,str(arrayRAS[row,col-1]))
                        tableNodeLPS.SetCellText(row,col,str(-arrayLPS[row,col-1]))
                        tableNodeLPS.SetCellText(row,col,str(-arrayLPS[row,col-1]))
                        tableNodeLPS.SetCellText(row,col,str(-arrayLPS[row,col-1]))
                    elif col==3:
                        tableNodeRAS.SetCellText(row,col,str(arrayRAS[row,col-1]))
                        tableNodeRAS.SetCellText(row,col,str(arrayRAS[row,col-1]))
                        tableNodeRAS.SetCellText(row,col,str(arrayRAS[row,col-1]))
                        tableNodeLPS.SetCellText(row,col,str(arrayLPS[row,col-1]))
                        tableNodeLPS.SetCellText(row,col,str(arrayLPS[row,col-1]))
                        tableNodeLPS.SetCellText(row,col,str(arrayLPS[row,col-1]))
            slicer.mrmlScene.AddNode(tableNodeRAS)
            slicer.mrmlScene.AddNode(tableNodeLPS)
    
#----Funzione per salvare le tabelle----#
def saveTableFunction(dirName, fileName):
    tables = slicer.mrmlScene.GetNodesByClass('vtkMRMLTableNode')
    for table in tables:
        if (table.GetColumnName(1)) == 'r': 
            filenameRAS = dirName + '/' + fileName + '-' + table.GetName() + '-RAS.txt'
            slicer.util.saveNode(table, filenameRAS)
        elif (table.GetColumnName(1)) == 'l':
            filenameLPS = dirName + '/' + fileName +  '-' + table.GetName() + '-LPS.txt'
            slicer.util.saveNode(table, filenameLPS)
            
#----Data Browser----#
def saveFunction():
    planes = slicer.mrmlScene.GetNodesByClass('vtkMRMLMarkupsPlaneNode')
    for plane in planes:
        plane.SetPlaneType(0)
    nodes = slicer.mrmlScene.GetNodesByClass('vtkMRMLMarkupsNode')
    for node in nodes:
        for i in range(0,node.GetNumberOfControlPoints()):
            node.SetNthControlPointLocked(i,True)
    saveCheckBox.connect('clicked(bool)', functionButton)
    saveWindow.setLayout(saveLayout)
    saveWindow.show()        
    
def placeMode(bool):
    interactionNode = slicer.mrmlScene.GetNodeByID("vtkMRMLInteractionNodeSingleton")
    if bool==True:
        placeModePersistence = 1
        interactionNode.SetPlaceModePersistence(placeModePersistence)
    else:
        placeModePersistence = 0
        interactionNode.SetPlaceModePersistence(placeModePersistence)

def createSimpleDataBrowser():    
    mainWindow = slicer.util.mainWindow()
    dockWidget = qt.QDockWidget("Data browser", mainWindow)
    dockWidget.setFeatures(qt.QDockWidget.DockWidgetClosable) # Permettere il riposizionamento della finestra: + qt.QDockWidget.DockWidgetMovable)
    dockWidget.setMinimumWidth(10)
    dockWidget.setMaximumWidth(570)
    mainFrame = qt.QFrame(dockWidget)    
    mainFrameLayout = qt.QVBoxLayout(mainFrame)  
    
    def dicomFunction(bool):
        dicomWidget.setVisible(bool)
        
#Add/Save data                
    collapsibleButton_Data = ctk.ctkCollapsibleButton()
    collapsibleButton_Data.text = 'Data'
    collapsibleButton_Data_Layout = qt.QFormLayout(collapsibleButton_Data)
    dicom = slicer.modules.dicom.widgetRepresentation()
    dicomWidget = qt.QWidget()
    dicomWidgetLayout = qt.QHBoxLayout()
    dicomWidgetLayout.addWidget(dicom)
    dicomWidget.setLayout(dicomWidgetLayout)
    dicomButton = qt.QPushButton('Load DICOM')
    dicomButton.setCheckable(True)
    collapsibleButton_Data_Layout.addRow(dicomButton)
    dicomButton.connect('clicked(bool)', dicomFunction)
    loadDataButton = qt.QPushButton('Load Data')
    collapsibleButton_Data_Layout.addRow(loadDataButton)
    loadDataButton.connect('clicked()', slicer.util.openAddDataDialog)
    saveDataButton = qt.QPushButton('Save Data')
    collapsibleButton_Data_Layout.addRow(saveDataButton)
    saveDataButton.connect('clicked()', saveFunction)  
    collapsibleButton_Data.collapsed = True    
#Add switch layout buttons
    collapsibleButton_Layout = ctk.ctkCollapsibleButton()
    collapsibleButton_Layout.text = 'Layout'    
    collapsibleButton_Layout_Layout = qt.QHBoxLayout(collapsibleButton_Layout)
    displayPresets = (
        (slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView, "LayoutFourUpView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpRedSliceView, "LayoutOneUpRedSliceView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpYellowSliceView, "LayoutOneUpYellowSliceView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpGreenSliceView, "LayoutOneUpGreenSliceView.png"),
        (slicer.vtkMRMLLayoutNode.SlicerLayoutOneUp3DView, "LayoutOneUp3DView.png"),
        )
    for (layoutId, iconName) in displayPresets:
        switchLayoutButton = qt.QPushButton("")
        switchLayoutButton.setIcon(qt.QIcon(":Icons/"+iconName))
        collapsibleButton_Layout_Layout.addWidget(switchLayoutButton, 0, 0)
        switchLayoutButton.connect('clicked()', lambda layoutId=layoutId: slicer.app.layoutManager().setLayout(layoutId))        
    collapsibleButton_Layout.collapsed = True
#Add markups buttons
    collapsibleButton_Markups = ctk.ctkCollapsibleButton()
    collapsibleButton_Markups.text = 'Markups'
    collapsibleButton_Markups_Form = qt.QFormLayout(collapsibleButton_Markups)
    collapsibleButton_Markups_Layout = qt.QHBoxLayout()
    checkBoxPlaceMode = qt.QCheckBox('Place multiple control points')
    markups = slicer.modules.markups.createNewWidgetRepresentation()
    markups.enter()
    slicer.util.findChild(markups, "createMarkupsGroupBox").hide()
    slicer.util.findChild(markups, "displayCollapsibleButton").hide()
    slicer.util.findChild(markups, "measurementsCollapsibleButton").hide()
    slicer.util.findChild(markups, "advancedCollapsibleButton").hide()
    slicer.util.findChild(markups, "exportImportCollapsibleButton").hide()
    slicer.util.findChild(markups, "planeSettingsCollapseButton").hide() 
    slicer.util.findChild(markups, "angleMeasurementModeCollapsibleButton").hide()
    groupBoxCreateMarkups = qt.QGroupBox()
    layoutCreateMarkups_H = qt.QHBoxLayout()
    layoutCreateMarkups_V = qt.QVBoxLayout()
    groupBoxCreateMarkups.setLayout(layoutCreateMarkups_V)
    fiducialPushButton = slicer.util.findChild(markups, "CreateFiducialPushButton")
    linePushButton = slicer.util.findChild(markups, "CreateLinePushButton")
    anglePushButton = slicer.util.findChild(markups, "CreateAnglePushButton")
    planePushButton = slicer.util.findChild(markups, "CreatePlanePushButton")
    planeType = slicer.util.findChild(markups, "planeTypeComboBox")
    checkBoxPlaceMode.connect('clicked(bool)', placeMode)
    layoutCreateMarkups_H.addWidget(fiducialPushButton)
    layoutCreateMarkups_H.addWidget(linePushButton)
    layoutCreateMarkups_H.addWidget(anglePushButton)
    layoutCreateMarkups_H.addWidget(planePushButton)
    layoutCreateMarkups_H.addWidget(planeType)
    layoutCreateMarkups_V.addLayout(layoutCreateMarkups_H)
    layoutCreateMarkups_V.addWidget(checkBoxPlaceMode)
    collapsibleButton_Markups_Layout.addWidget(markups)
    collapsibleButton_Markups_Form.addRow(groupBoxCreateMarkups)
    collapsibleButton_Markups_Form.addRow(collapsibleButton_Markups_Layout)
    collapsibleButton_Markups.collapsed = True 
#Add Texts
    collapsibleButton_Texts = ctk.ctkCollapsibleButton()
    collapsibleButton_Texts.text = 'Texts' 
    collapsibleButton_Texts_Form = qt.QFormLayout(collapsibleButton_Texts)
    collapsibleButton_Texts_Layout = qt.QHBoxLayout()  
    text = slicer.modules.texts.createNewWidgetRepresentation()  
    slicer.util.findChild(text, "CollapsibleButton").hide()   
    collapsibleButton_Texts_Layout.addWidget(text)
    collapsibleButton_Texts_Form.addRow(collapsibleButton_Texts_Layout)
    collapsibleButton_Texts.collapsed = True       
#Scroll area
    area = qt.QScrollArea()
    area.setWidgetResizable(True)
    widget = qt.QWidget()
    area.setWidget(widget)
    layoutWidget = qt.QVBoxLayout()
    widget.setLayout(layoutWidget)      
    layoutWidget.addWidget(collapsibleButton_Data)
    layoutWidget.addWidget(dicomWidget)
    layoutWidget.addWidget(collapsibleButton_Layout)
    layoutWidget.addWidget(collapsibleButton_Markups)
    layoutWidget.addWidget(collapsibleButton_Texts)
    dicomWidget.setVisible(False)
    layoutWidget.addStretch(1)
    mainFrameLayout.addWidget(area)
#Add data tree
    dataTreeWidget = slicer.qMRMLSubjectHierarchyTreeView(mainFrame)
    dataTreeWidget.setMRMLScene(slicer.mrmlScene)
    dataTreeWidget.setColumnHidden(dataTreeWidget.model().idColumn, True)
    dataTreeWidget.setColumnHidden(dataTreeWidget.model().transformColumn, True)
    dataTreeWidget.setMaximumHeight(300)
    dataTreeWidget.setMinimumWidth(150)
    mainFrameLayout.addWidget(dataTreeWidget) 
    dockWidget.setWidget(mainFrame)
    mainWindow.addDockWidget(qt.Qt.LeftDockWidgetArea, dockWidget)       
#Load Data Browser:
createSimpleDataBrowser()

slicernb.AppWindow()